## ***SACHAROMYCES CEREVISIAE*** OPTIMIZATION OUTPUT DATA PROCESSING

Since we've runned 10 Gene Over/Under expression optimization problem we've quite a lot of results to carefully interpret. We'll be using pandas (a powerful python library to handle data science preprocessing stages). <br> First we'll merge all 10 .csv file into 1 .csv file to better understand our data and preform various preprocessing steps to gather only the best solutions possible for our problem.

In [145]:
import os
import glob
import pandas as pd
import warnings

warnings.filterwarnings("ignore")

# pandas has a set_option method that let you costumize the behaviour and display of dataframes
# here the set_option method is set to display the full contents of our dataframe
# pd.set_option("display.max_rows", None)
# pd.options.display.max_rows = 15

# setting the path to join multiple files 
files = os.path.join(r'C:\Users\geral\Desktop\BIOINF_PROJECT\2-PEopt\src\outputGOU', "*.csv")

# return a list of the merged files
files = glob.glob(files)

# using pandas methods concat and read_csv to join files
df = pd.concat([pd.read_csv(f, sep=";", index_col=False) for f in files], ignore_index=True)
print(f"This dataframe has: {df.shape[0]} rows and {df.shape[1]} columns.")

# pandas dataframe manipulation to drop unnecessary columns (columns only with NaN values)
df.drop(["Unnamed: 3", "solution"], axis=1, inplace=True)
print(f"This dataframe has: {df.shape[0]} rows and {df.shape[1]} columns after manipulation.")

# change column names
df.rename(columns={"Unnamed: 4": "Genes", "Unnamed: 6": "Reactions"}, inplace=True)


This dataframe has: 189 rows and 7 columns.
This dataframe has: 189 rows and 5 columns after manipulation.


In [146]:
# cell to display part of out dataframe
display(df)

,BPCY (r_4041 * r_1589),TargetFlux r_4041 with at least 0.0 of biomass (None),WYIELD biomass: r_4041 product: r_1589,Genes,Reactions
0,0.000000,0.000000,0.000000,"{'YBR166C': 0.25, 'YNL220W': 0.125, 'YIL043C':...","{'r_0714': 0, 'r_0020': (2.6153065645050235, 1..."
1,0.000000,0.000000,0.000000,"{'YBR166C': 0.25, 'YNL220W': 0.125, 'YGL063W':...","{'r_0714': 0, 'r_0020': (2.6153065645050084, 1..."
2,0.000000,0.000000,0.000000,"{'YDR305C': 0.5, 'YGR208W': 0.25, 'YOL126C': 0...","{'r_1025': (0, 0), 'r_0714': 0, 'r_0138': (0.0..."
3,0.000000,0.000000,0.000000,"{'YDR305C': 0.5, 'YNL241C': 4, 'YGR208W': 0.25...","{'r_1025': (0, 0), 'r_0714': 0, 'r_0138': (0.0..."
4,0.000000,0.000000,0.000000,"{'YDR305C': 0.5, 'YNL241C': 4, 'YGR208W': 0.25...","{'r_1025': (0, 0), 'r_0714': 0, 'r_0138': (0.0..."
...,...,...,...,...,...
184,0.650626,0.598086,0.351609,"{'YMR267W': 2, 'YLR308W': 2, 'YNL331C': 0.25, ...","{'r_1069': 0, 'r_0468': (0, 0), 'r_1109': (0, ..."
185,0.249412,0.094355,1.269405,"{'YJR103W': 16, 'YJL121C': 4, 'YFL058W': 0.125...","{'r_0279': (2.644694430440478, 10000), 'r_0307..."
186,0.090259,0.094355,1.319482,"{'YPL262W': 2, 'YGR193C': 0.125, 'YJL121C': 2,...","{'r_0279': (2.6446944304404685, 10000), 'r_045..."
187,0.054513,0.094355,1.265827,"{'YPL262W': 2, 'YNL191W': 0, 'YGL148W': 4, 'YF...","{'r_4171': (0, 0), 'r_0279': (1.32234721522023..."


Our first step to achieve a desirable solution is to drop all rows that doesn't correlate with our objetive: Biomass > 0.1 <br> So let's drop some rows and see what we have.

In [147]:
df.drop(df[df["BPCY (r_4041 * r_1589)"] < 0.1].index, inplace=True)
print(f"This dataframe has {df.shape[0]} rows and {df.shape[1]} columns after deleting all rows with biomass inferior to 0.1.")
df.reset_index(drop=True, inplace=True)

This dataframe has 115 rows and 5 columns after deleting all rows with biomass inferior to 0.1.


Our goal now is to reduce even more our pool of possible solutions by choosing the ones that have less genetic modifications. For that we will convert the dataframe solution values to a dictionary and from there we will be able to check the ones that are more suitable within our main goal. <br>
To make this analysis we need to make some changes to our dataframe first. Although our "Genes" and "Reactions" column values look like a dictionary they're actually strings formatted as dictionaries. To make these changes we will use **ast**, a python library that makes transforming a dictionary formatted string into a python dictionary object possible.

In [148]:
df.dtypes

BPCY (r_4041 * r_1589)                                   float64
TargetFlux r_4041 with at least 0.0 of biomass (None)    float64
WYIELD biomass: r_4041 product: r_1589                   float64
Genes                                                     object
Reactions                                                 object
dtype: object

In [149]:
import ast

df["Reactions"] = df["Reactions"].apply(ast.literal_eval)
df["Genes"] = df["Genes"].apply(ast.literal_eval)


In [156]:
df.drop(df[df["Reactions"].map(len) > 5].index & df[df["Genes"].map(len) > 5].index, inplace=True)
df.reset_index(drop=True, inplace=True)

In order to analyse the natural maximum 2-phenylethanol production capabilities of *Sacharomyces cerevisiae* we perform a FVA analysis on the YEAST8 model using MEWpy

In [151]:
from cobra.io import read_sbml_model
from mewpy.simulation import get_simulator

model = read_sbml_model("models/yeast-GEM.xml")

O2 = "r_1992" # EX_o2_e (r_1992)
GLC = "r_1714" # EX_glc__D_e (r_1714)

envcond = {
        O2: (-20.0, 100000.0), 
        GLC: (-10.0, 100000.0)
        }

PRODUCT_ID = "r_1589" # EX_2phetoh_e (r_1589)

simul = get_simulator(model, envcond=envcond)
print(simul.FVA(reactions = [PRODUCT_ID], format = 'df'))

  Reaction ID  Minimum   Maximum
0      r_1589      0.0  0.927562


However, by analysing our optimization results within our dataframe we can actually see that there are a set of modifications that increases the *Sacharomyces cerevisiae* natural production capabilities of 2-phenylethanol by almost 4 times.

In [157]:
print(f"Minimum FVA: {df['WYIELD biomass: r_4041 product: r_1589 '].min()}")
print(f"Maximum FVA: {df['WYIELD biomass: r_4041 product: r_1589 '].max()}")

Minimum FVA: 0.720417902558721
Maximum FVA: 3.61196810332161


In [158]:
# get the row which corresponds with our maximum level of WYIELD
df.loc[df["WYIELD biomass: r_4041 product: r_1589 "] == df["WYIELD biomass: r_4041 product: r_1589 "].max()]


,BPCY (r_4041 * r_1589),TargetFlux r_4041 with at least 0.0 of biomass (None),WYIELD biomass: r_4041 product: r_1589,Genes,Reactions
15,0.475955,0.118863,3.611968,"{'YGR256W': 2, 'YNL316C': 32, 'Q0045': 0.25}","{'r_0438': (0, 19.307971581669968), 'r_0889': ..."


In [159]:
print(f"Genes: {df['Genes'][15]}")
print(f"Reactions: {df['Reactions'][15]}")

Genes: {'YGR256W': 2, 'YNL316C': 32, 'Q0045': 0.25}
Reactions: {'r_0438': (0, 19.307971581669968), 'r_0889': (7.06883478546144, 10000), 'r_0938': (3.8707169858806947, 10000)}
